# Introduction and Schema Diagram

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

# Overview of the Data

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [10]:
%%sql
SELECT * FROM customer
LIMIT 5;

Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


In [11]:
%%sql
SELECT * FROM invoice_line
LIMIT 5;

Done.


invoice_line_id,invoice_id,track_id,unit_price,quantity
1,1,1158,0.99,1
2,1,1159,0.99,1
3,1,1160,0.99,1
4,1,1161,0.99,1
5,1,1162,0.99,1


# Selecting New Albums to Purchase

In [5]:
%%sql
WITH usa_tracks_sold AS
    (
    SELECT
        country,
        c.customer_id customer_id,
        invoice_line_id,
        track_id
    FROM customer c
    INNER JOIN invoice i ON i.customer_id = c.customer_id
    INNER JOIN invoice_line il ON il.invoice_id = i.invoice_id
    WHERE country = 'USA'
    ),    
    g_ts_usa AS
    (
    SELECT
        g.name genre_name,
        COUNT(invoice_line_id) tracks_sold
    FROM usa_tracks_sold uts
    INNER JOIN track t ON t.track_id = uts.track_id
    INNER JOIN genre g ON g.genre_id = t.genre_id
    GROUP BY genre_name
    )
    
SELECT
    genre_name,
    tracks_sold,
    ROUND(CAST(tracks_sold AS Float) / (SELECT COUNT(*)
                   FROM usa_tracks_sold), 3) AS pct_tracks_sold
FROM g_ts_usa
ORDER BY 2 DESC;

Done.


genre_name,tracks_sold,pct_tracks_sold
Rock,561,0.534
Alternative & Punk,130,0.124
Metal,124,0.118
R&B/Soul,53,0.05
Blues,36,0.034
Alternative,35,0.033
Latin,22,0.021
Pop,22,0.021
Hip Hop/Rap,20,0.019
Jazz,14,0.013


Above, we can see the genres ordered by the number of tracks sold. Based on sales of tracks from each genres, we can see that Punk, Blues and Pop come ahead, so,in order, our recommendation is:

- Red Tone (Punk)
- Slim Jim Bites (Blues)
- Meteor and the Girls (Pop)

However, it's important to highlight the Rock genre, which represents 53.4% of sales, whereas the three genres from the deal sum only almost 18% of Chinook sales. A sugestion would be looking for Rock artists.

# Analyzing Employee Sales Performance

In [6]:
%%sql
WITH total_sales_support_agent AS
    (
    SELECT
        e.first_name || ' ' || e.last_name employee_name,
        *
    FROM invoice i
    INNER JOIN customer c ON c.customer_id = i.customer_id
    INNER JOIN employee e ON e.employee_id = c.support_rep_id
    )
    
SELECT 
    employee_name,
    ROUND(SUM(total), 2) total_sales,
    title,
    reports_to,
    hire_date,
    country
FROM total_sales_support_agent
GROUP BY employee_id
ORDER BY 2 DESC;

Done.


employee_name,total_sales,title,reports_to,hire_date,country
Jane Peacock,1731.51,Sales Support Agent,2,2017-04-01 00:00:00,United Kingdom
Margaret Park,1584.0,Sales Support Agent,2,2017-05-03 00:00:00,USA
Steve Johnson,1393.92,Sales Support Agent,2,2017-10-17 00:00:00,Canada


Blindly, we could say that the seller Steve Johnson is underperforming, as he is under 20% in sales in comparison to Jane Peacock. However, we can notice the `hire_date` difference, it's more than six months between these two seller, a relevant period of time to be put on the count.

# Analyzing Sales by Country

In [7]:
%%sql
WITH customer_invoice AS
    (
    SELECT 
        CASE
            WHEN(
                SELECT COUNT(*)
                FROM customer
                WHERE country=c.country) = 1 THEN 'Other'
            ELSE c.country
        END AS country_or_other,
        c.customer_id cust,
        *
    FROM customer c
    INNER JOIN invoice i ON i.customer_id = c.customer_id
    )
    
SELECT
    country_or_other,
    total_n_of_customers,
    total_value_of_sales,
    avg_sales_per_cust,
    avg_order_value
FROM
    (
    SELECT
        country_or_other,
        COUNT(DISTINCT cust) 'total_n_of_customers',
        SUM(total) 'total_value_of_sales',
        SUM(total) / COUNT(DISTINCT cust) 'avg_sales_per_cust',
        SUM(total) / COUNT(DISTINCT invoice_id) 'avg_order_value',
        CASE
            WHEN country_or_other = 'Other' THEN 1
            ELSE 0
        END AS sort
    FROM customer_invoice
    GROUP BY country_or_other
    ORDER BY sort ASC, total_value_of_sales DESC
    );

Done.


country_or_other,total_n_of_customers,total_value_of_sales,avg_sales_per_cust,avg_order_value
USA,13,1040.4899999999998,80.0376923076923,7.942671755725189
Canada,8,535.5900000000001,66.94875000000002,7.047236842105265
Brazil,5,427.68000000000006,85.53600000000002,7.0111475409836075
France,5,389.0699999999999,77.81399999999998,7.781399999999998
Germany,4,334.62,83.655,8.161463414634147
Czech Republic,2,273.24000000000007,136.62000000000003,9.108000000000002
United Kingdom,3,245.52,81.84,8.768571428571429
Portugal,2,185.13000000000002,92.56500000000001,6.383793103448276
India,2,183.14999999999998,91.57499999999999,8.72142857142857
Other,15,1094.9400000000003,72.99600000000002,7.44857142857143


# Albums vs Individual Tracks

In [8]:
%%sql
WITH invoice_first_track AS
    (
     SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY 1
    )

SELECT
    album_purchase,
    COUNT(invoice_id) number_of_invoices,
    CAST(count(invoice_id) AS FLOAT) / (
                                         SELECT COUNT(*) FROM invoice
                                      ) percent
FROM
    (
    SELECT
        ifs.*,
        CASE
            WHEN
                 (
                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 

                  EXCEPT 

                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id
                 ) IS NULL
             AND
                 (
                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id

                  EXCEPT 

                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 
                 ) IS NULL
             THEN "yes"
             ELSE "no"
         END AS "album_purchase"
     FROM invoice_first_track ifs
    )
GROUP BY album_purchase;

Done.


album_purchase,number_of_invoices,percent
no,500,0.8143322475570033
yes,114,0.18566775244299674


Album purchases account for 18.6% of purchases. Based on this data, I would recommend against purchasing only select tracks from albums from record companies, since there is potential to lose one fifth of revenue.